In [2]:
import pandas as pd
import numpy as np
import datetime

from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.sql import func
from datetime import datetime
from flask_login import UserMixin
from sqlalchemy.sql import func

In [3]:
db = SQLAlchemy()  # Create a new SQLAlchemy instance

split_workout = db.Table('split_workout',
    db.Column('split_id', db.Integer, db.ForeignKey('split.id'), primary_key=True),
    db.Column('workout_id', db.Integer, db.ForeignKey('workout.id'), primary_key=True)
)

class Note(db.Model):
    id = db.Column(db.Integer, primary_key = True)
    data = db.Column(db.String(10000))
    date = db.Column(db.DateTime(timezone = True), default = func.now)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))

class Workout(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    workout_name = db.Column(db.String(10000))
    num_movements = db.Column(db.Integer)
    movements = db.relationship('Movement', backref='workout', lazy=True)
    splits = db.relationship('Split', secondary=split_workout, backref=db.backref('workouts', lazy='dynamic'))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))  

class Movement(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    mov_name = db.Column(db.String(100))
    reps = db.Column(db.Integer)
    sets = db.Column(db.Integer)
    workout_id = db.Column(db.Integer, db.ForeignKey('workout.id'))

class Split(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    split_name = db.Column(db.String(10000))
    is_active = db.Column(db.Boolean)
    # workouts = db.relationship('Workout', secondary=split_workout, lazy='dynamic', backref=db.backref('splits', lazy=True))
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))

class WorkoutData(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    workout_id = db.Column(db.Integer, db.ForeignKey('workout.id'))
    movement_id = db.Column(db.Integer, db.ForeignKey('movement.id'))
    date = db.Column(db.DateTime(timezone = True), default = func.now)
    weight = db.Column(db.Integer, primary_key=True)
    reps = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key = True)
    email = db.Column(db.String(150), unique = True)
    password = db.Column(db.String(150))
    first_name = db.Column(db.String(150))
    notes = db.relationship('Note')
    workouts = db.relationship('Workout')
    splits = db.relationship('Split')



In [4]:
new_user = User(email = 'pranav@gmail.com', password = 'qwertyuiop', first_name = 'pranav')

In [7]:
test_df = pd.read_csv('workout_splits_1.csv')
test_df

,Workout Name,Movement Name,Reps,Sets
0,Push Day,Bench Press,5,5
1,Push Day,Shoulder Press,4,8
2,Push Day,Tricep Extensions,3,10
3,Pull Day,Pull Ups,4,8
4,Pull Day,Barbell Rows,4,6
5,Pull Day,Bicep Curls,3,10
6,Leg Day,Squat,5,5
7,Leg Day,Hamstring Curls,4,8
8,Leg Day,Calf Raises,3,10


In [8]:
push_day = Workout(id = 1, 
                   workout_name = 'Push Day', 
                   num_movements = 3)

pull_day = Workout(id = 2, 
                   workout_name = 'Pull Day', 
                   num_movements = 3)

leg_day = Workout(id = 3, 
                   workout_name = 'Leg Day', 
                   num_movements = 3)

workouts = [push_day, pull_day, leg_day]
workouts

[<Workout (transient 5101930256)>,
 <Workout (transient 5101552336)>,
 <Workout (transient 4359248848)>]

In [13]:
new_user.workouts = workouts

In [9]:
count = 1
for workout in workouts:
    df = test_df[test_df['Workout Name'] == workout.workout_name].reset_index()
    movements = []
    for i in range(len(df)):
        movements.append(Movement(mov_name = df.loc[i]['Movement Name'], 
                                  reps = df.loc[i]['Reps'], 
                                  sets = df.loc[i]['Sets'],
                                  id = count))
    workout.movements = movements
    count += 1
    

In [19]:
workout = new_user.workouts[0]

template = pd.DataFrame(columns = ['Movement', 'Weight', 'Reps', 'Set'])
count = 0
for movement in workout.movements:
    for i in range(movement.sets):
        template.loc[count] = [movement.mov_name, 0, 0, i]
        count += 1
template_export = template.to_csv()

In [34]:
template_export

',Movement,Weight,Reps,Set\n0,Bench Press,0,0,0\n1,Bench Press,0,0,1\n2,Bench Press,0,0,2\n3,Bench Press,0,0,3\n4,Bench Press,0,0,4\n5,Shoulder Press,0,0,0\n6,Shoulder Press,0,0,1\n7,Shoulder Press,0,0,2\n8,Shoulder Press,0,0,3\n9,Shoulder Press,0,0,4\n10,Shoulder Press,0,0,5\n11,Shoulder Press,0,0,6\n12,Shoulder Press,0,0,7\n13,Tricep Extensions,0,0,0\n14,Tricep Extensions,0,0,1\n15,Tricep Extensions,0,0,2\n16,Tricep Extensions,0,0,3\n17,Tricep Extensions,0,0,4\n18,Tricep Extensions,0,0,5\n19,Tricep Extensions,0,0,6\n20,Tricep Extensions,0,0,7\n21,Tricep Extensions,0,0,8\n22,Tricep Extensions,0,0,9\n'

In [20]:
template_export

',Movement,Weight,Reps,Set\n0,Bench Press,0,0,0\n1,Bench Press,0,0,1\n2,Bench Press,0,0,2\n3,Bench Press,0,0,3\n4,Bench Press,0,0,4\n5,Shoulder Press,0,0,0\n6,Shoulder Press,0,0,1\n7,Shoulder Press,0,0,2\n8,Shoulder Press,0,0,3\n9,Shoulder Press,0,0,4\n10,Shoulder Press,0,0,5\n11,Shoulder Press,0,0,6\n12,Shoulder Press,0,0,7\n13,Tricep Extensions,0,0,0\n14,Tricep Extensions,0,0,1\n15,Tricep Extensions,0,0,2\n16,Tricep Extensions,0,0,3\n17,Tricep Extensions,0,0,4\n18,Tricep Extensions,0,0,5\n19,Tricep Extensions,0,0,6\n20,Tricep Extensions,0,0,7\n21,Tricep Extensions,0,0,8\n22,Tricep Extensions,0,0,9\n'

In [22]:
test_data = pd.read_csv('workout_data_1.csv')
test_data

,Movement,Weight,Reps,Set
0,Bench Press,135,5,1
1,Bench Press,145,5,2
2,Bench Press,155,5,3
3,Bench Press,160,5,4
4,Bench Press,165,5,5
5,Shoulder Press,20,8,1
6,Shoulder Press,25,8,2
7,Shoulder Press,30,8,3
8,Shoulder Press,35,8,4
9,Tricep Extensions,10,10,1


In [29]:
[el.lower() for el in test_data.columns.tolist()]

['movement', 'weight', 'reps', 'set']

In [23]:
bruh = test_data.loc[0]

In [30]:
bruh['Movement']

'Bench Press'

In [32]:
[mov.mov_name for mov in workout.movements] == test_data['Movement'].unique().tolist()

True

In [33]:
datetime.now()

datetime.datetime(2024, 1, 10, 23, 57, 13, 575533)

In [35]:
ass = 'template.csv'

ass[-4:]

'.csv'